In [1]:
#필요한 모듈 임포트하기
import pandas as pd
from pandas import DataFrame  as df
import numpy as np
from collections import Counter
from konlpy.tag import Okt
okt = Okt()
%matplotlib inline
import matplotlib.pyplot as plt
import re 
import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel
from datetime import datetime
import os


In [2]:
Data = pd.read_csv('./DATA/Seoul_2015.csv', encoding='utf8')

In [3]:
Data

,time,total
0,2015-03-01 0:27,수원에 있는 성대 대학원까지 서울대 졸업한 애들이 오는 이유가 뭐냐 안 불편하나
1,2015-03-01 0:51,공대생들이거봐라 니들 구글코리아 신입으로 입사하고 막 그러냐구글코리아 들어간사람있냐...
2,2015-03-01 1:47,어제오늘 일해서 백만원 벌었다 기쁘다 한학기 등록금 벌음
3,2015-03-01 2:38,사기내 다 쓸어버리네
4,2015-03-01 6:10,잠때문에 죽겄다 학기중에도 밤에 잠못자고 낮에 졸린것 때문에 고생했는데방학하니까 자...
...,...,...
7717,2015-09-27 12:27,신환회비 너무 아까운 부분 새내기들 철저하게 밥 안사주기로 결심
7718,2015-09-27 12:27,나랑 노래 배틀 뜰 새끼 구한다 코인노래방에서 붙자덤벼
7719,2015-09-27 12:27,지금 일반인 개방 아예 안되어있는 도서관 신중도밖에없 관정도 맘만먹으면 타대생 올 ...
7720,2015-09-27 12:27,신입생 지원품 패키지이게 머에얌 궁금궁금


# 입력 받은 단어와 기간으로 데이터 처리

In [4]:
Data['time'] = pd.to_datetime(Data["time"])

# 일자 별 문서 수 

In [5]:
element_count = {}
for item in Data['time']:
    element_count.setdefault(item, 0)
    element_count[item] += 1
doc_count = pd.DataFrame.from_dict(element_count, orient='index', columns=["doc_count"])

Data.total = Data.total.astype(str)

clean_Data = Data
clean_Data

,time,total
0,2015-03-01 00:27:00,수원에 있는 성대 대학원까지 서울대 졸업한 애들이 오는 이유가 뭐냐 안 불편하나
1,2015-03-01 00:51:00,공대생들이거봐라 니들 구글코리아 신입으로 입사하고 막 그러냐구글코리아 들어간사람있냐...
2,2015-03-01 01:47:00,어제오늘 일해서 백만원 벌었다 기쁘다 한학기 등록금 벌음
3,2015-03-01 02:38:00,사기내 다 쓸어버리네
4,2015-03-01 06:10:00,잠때문에 죽겄다 학기중에도 밤에 잠못자고 낮에 졸린것 때문에 고생했는데방학하니까 자...
...,...,...
7717,2015-09-27 12:27:00,신환회비 너무 아까운 부분 새내기들 철저하게 밥 안사주기로 결심
7718,2015-09-27 12:27:00,나랑 노래 배틀 뜰 새끼 구한다 코인노래방에서 붙자덤벼
7719,2015-09-27 12:27:00,지금 일반인 개방 아예 안되어있는 도서관 신중도밖에없 관정도 맘만먹으면 타대생 올 ...
7720,2015-09-27 12:27:00,신입생 지원품 패키지이게 머에얌 궁금궁금


In [6]:
#데이터 프레임의 'text' 열의 값 중 keyword1이나 keyword 2가 포함된 행은 삭제
#clean_Data = Data[~Data['text'].str.contains('keyword1|keyword2')]

#text와 timestamp 열을 기준으로 중복된 데이터를 삭제, inplace : 데이터 프레임을 변경할지 선택(원본을 대체)
clean_Data.drop_duplicates(subset=['total','time'], inplace=True)

#한글이 아니면 빈 문자열로 바꾸기
clean_Data['total'] = clean_Data['total'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]',' ',regex=True)

#빈 문자열 NAN 값으로 바꾸기
clean_Data = clean_Data.replace({'': np.nan})
clean_Data = clean_Data.replace(r'^\s*$', None, regex=True)

#NAN 이 있는 행은 삭제
clean_Data.dropna(how='any', inplace=True)

#인덱스 차곡차곡
clean_Data = clean_Data.reset_index(drop=True)

print(clean_Data.isnull().values.any()) 


#텍스트 데이터를 리스트로 변환
Data_list = clean_Data.total.values.tolist()




#리스트를 요소별로(트윗 하나) 가져와서 명사만 추출한 후 리스트로 저장
data_word=[]
for i in range(len(Data_list)):
    try:
        data_word.append(okt.nouns(Data_list[i]))
    except Exception as e:
        continue
Data['clean'] = data_word

c:\users\yss\appdata\local\programs\python\python35\lib\site-packages\pandas\core\missing.py:48: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask = arr == x


False


In [8]:
id2word=corpora.Dictionary(data_word)
id2word.filter_extremes(no_below = 20)
texts = data_word
corpus=[id2word.doc2bow(text) for text in texts]

# os.environ['MALLET_HOME'] = 'C:/Users/YSS/Downloads/mallet-2.0.8'

mallet_path = 'D:/mallet-2.0.8/bin/mallet'
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=10, id2word=id2word)


coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=texts, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()


def compute_coherence_values(dictionary, corpus, texts, limit, start=4, step=2):

    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=data_word, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values



# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=texts, start=4, limit=21, step=2)







In [ ]:
limit=21; start=4; step=2;
x = range(start, limit, step)
topic_num = 0
count = 0
max_coherence = 0
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", cv)
    coherence = cv
    if coherence >= max_coherence:
        max_coherence = coherence
        topic_num = m
        model_list_num = count   
    count = count+1

        
# Select the model and print the topics
optimal_model = model_list[model_list_num]
model_topics = optimal_model.show_topics(formatted=False)
#print(optimal_model.print_topics(num_words=10))








def format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    #ldamodel[corpus]: lda_model에 corpus를 넣어 각 토픽 당 확률을 알 수 있음
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num,topn=10)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
    print(type(sent_topics_df))

    # Add original text to the end of the output
    #contents = pd.Series(texts)
    #sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    sent_topics_df = pd.concat([sent_topics_df, Data['total'],Data['time'],Data['clean']], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=Data_list)

# Format
df_topic_tweet = df_topic_sents_keywords.reset_index()
df_topic_tweet.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'total','Time','Clean']

# Show각 문서에 대한 토픽
#df_dominant_topic=df_dominant_topic.sort_values(by=['Dominant_Topic'])
#df_topic_tweet








# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)


topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()
topic_counts.sort_index(inplace=True)

topic_contribution = round(topic_counts/topic_counts.sum(), 4)
topic_contribution

lda_inform = pd.concat([sent_topics_sorteddf_mallet, topic_counts, topic_contribution], axis=1)
lda_inform.columns=["Topic_Num", "Topic_Perc_Contrib", "Keywords", "total", "time","Clean","Num_Documents", "Perc_Documents"]
lda_inform = lda_inform[["Topic_Num","Keywords","Num_Documents","Perc_Documents"]]
lda_inform
#lda_inform.Topic_Num = lda_inform.Topic_Num.astype(int)
lda_inform['Topic_Num'] =lda_inform['Topic_Num'] +1
lda_inform.Topic_Num = lda_inform.Topic_Num.astype(str)
lda_inform['Topic_Num'] =lda_inform['Topic_Num'].str.split('.').str[0]
df_topic_tweet['Dominant_Topic'] =df_topic_tweet['Dominant_Topic'] +1
df_topic_tweet.Dominant_Topic = df_topic_tweet.Dominant_Topic.astype(str)
df_topic_tweet['Dominant_Topic'] =df_topic_tweet['Dominant_Topic'].str.split('.').str[0]


# 토픽 별 트윗 퍼센트

In [ ]:
# lda_inform.to_csv ("./Result/lda_inform.csv", index = None)
lda_inform

In [ ]:
#df_topic_tweet.to_csv ("./Result/df_topic_tweet.csv", index = None)
df_topic_tweet

# 토픽 별 트윗 저장

In [ ]:
#토픽별 트윗 저장
for i in range(1,topic_num+1):
    globals()['df_{}'.format(i)]=df_topic_tweet.loc[df_topic_tweet.Dominant_Topic==str(i)]
    globals()['df_{}'.format(i)].sort_values('Topic_Perc_Contrib',ascending=False,inplace = True)
    globals()['df_{}'.format(i)].to_csv ("./Result/topic("+str(i)+")_tweet.csv", index = None)
    



In [ ]:
#df_1~df_?? 토픽 수 까지
df_1

# 토픽 별 word cloud

In [ ]:
for i in range(1,topic_num+1):
    #data_list = globals()['df_{}'.format(i)].Text.values.tolist()
    long_string = sum(globals()['df_{}'.format(i)].Clean.values,[])
    str(long_string)
    
    
    
    #data_word=[x for x in data_word if not x.isdigit()]
    
    freq=pd.Series(long_string).value_counts().head(50)
    freq=dict(freq)
    # Create a WordCloud object
    freq=dict(freq)
    
    wordcloud = WordCloud(font_path="./Font/BMHANNA_11yrs_ttf.ttf",
             relative_scaling = 0.2,
             background_color = 'white',
            ).generate_from_frequencies(freq)
    
    # Visualize the word cloud
    wordcloud.to_image()
    plt.figure(figsize=(16,8))
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()

    plt.savefig("./Result/topic("+str(i)+")wordcloud.png")


# 토픽 별 긍부정 비율

In [ ]:

for i in range(1,topic_num+1):
    globals()['df_{}_pn'.format(i)]=globals()['df_{}'.format(i)].label.value_counts(normalize=True) * 100
    globals()['df_{}_pn'.format(i)]
  #  globals()['df_{}_pn'.format(i)].to_csv ("./Result/topic_tweet(topic"+str(i)+")posneg.csv", index = None)
    



In [ ]:
#df_1_pn~df_?? 토픽 수 까지
df_1_pn


# 토픽 별 긍정 부정 트윗 

In [ ]:
for i in range(1, topic_num+1):
    globals()['df_{}_positive'.format(i)]=globals()['df_{}'.format(i)][globals()['df_{}'.format(i)]['label'] == 1]
    globals()['df_{}_negative'.format(i)]=globals()['df_{}'.format(i)][globals()['df_{}'.format(i)]['label'] == 0]
    globals()['df_{}_positive'.format(i)].to_csv ("./Result/topic("+str(i)+")_tweet_positive.csv", index = None)
    globals()['df_{}_negative'.format(i)].to_csv ("./Result/topic("+str(i)+")_tweet_negative.csv", index = None)

In [ ]:
df_1_positive

In [ ]:
df_1_negative

In [ ]:
Data= pd.read_csv('./KOREA_Tweet_Data.csv', encoding="utf-8")